## Topic modeling: ADL (Latent Drichilet Allocation)

In [2]:
import pandas as pd
import gensim #the library for Topic modelling
from gensim.models.ldamulticore import LdaMulticore
from gensim import corpora, models
import pyLDAvis.gensim #LDA visualization library
from gensim import corpora

from nltk.corpus import stopwords
import string
from nltk.stem.wordnet import WordNetLemmatizer

import warnings
warnings.simplefilter('ignore')
from itertools import chain

from music21 import*
import numpy as np
import os
import itertools
from collections import OrderedDict
from operator import itemgetter
import operator
from heapq import nlargest
import re 
import csv
import matplotlib
import matplotlib.pyplot as plt
from heapq import nlargest
import glob

import nltk as nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
from IPython.display import HTML
from nltk.corpus import stopwords
import string
from nltk.stem.wordnet import WordNetLemmatizer
import warnings
warnings.simplefilter('ignore')
from itertools import chain
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import collections
from music21 import text, stream
import random
from itertools import chain
from collections import Counter

In [45]:
sw=['a','à','á','ah','ali','bi', 'bil', 'bila', 'bile', 'bili', 'bilo', 'biti', 'blizu', 'bo', 'bodo', 'bojo', 'bolj',
                  'bom', 'bomo', 'boste', 'bova', 'boš', 'brez', 'cel', 'cela', 'celi', 'celo', 'da','de' ,'do', 'dokler',
                  'dva','dve','dolcissimo','eden', 'en','et', 'eni', 'ena', 'ene', 'eno', 'etc', 'espressivo','g.', 'g','ga.', 'ga', 'i', 'idr', 'idr.','itd.', 
                  'itd', 'ii', 'iii','imam','ima','imava','imata','imamo','imajo','imaš','imeti', 'in', 'iv', 'ix', 'iz', 'jaz', 'je', 'ji', 'jih', 'jim', 'jo','ju', 'k','kadar', 'kadarkoli', 'kaj',
                  'kajti', 'kako', 'kakor', 'kamor', 'kamorkoli', 'kar', 'karkoli', 'katerikoli', 'kdaj', 'kdo', 'kdorkoli',
                  'ker', 'ki', 'kje', 'kjer', 'kjerkoli', 'knjim','ko', 'koder', 'koliko','kolikšen','kolikšno','kolikšna','koderkoli', 'koga', 'komu', 'kot', 'kratek', 
                  'kratka', 'kratke', 'kratki','la','le', 'mano','meni','me', 'med', 'medtem', 'mene', 'mi', 'midva', 'midve', 'mnogo', 'moje', 'moj','moja','mojih','mojemu','moji',
                  'mojima','mojega',"moj'ga",'moja','mojim','mu',
                  'na', 'nad', 'naj', 'najina', 'najino', 'naju', 'nas', 'nato', 'naš', 'naša', 'naše', 'ne', 'nedavno',
                  'nek', 'neka', 'nekaj', 'nekatere', 'nekateri', 'nekatero', 'nekdo', 'neke', 'nekega', 'neki', 'nekje',
                  'neko', 'nekoga', 'ni', 'nikamor', 'nikjer', 'nje', 'njega', 'nje', 'njegova', 'njegovo', 'njej', 'njemu',
                  'njen', 'njena', 'njegova', 'njegovih','njegovo','njegovega','njeno', 'nji', 'njih', 'njihov', 'njihova', 'njihovo', 'njiju', 'njim', 'njo', 'njun',
                  'njuna','nobena','nobene','nobeni','nobeno','noben','nobenega','nobenem','nobenemu','nobenih','nobeni',
                  'nobenih','njuno', 'no', 'npr.', 'o', 'ob', 'oba', 'obe', 'oboje', 'od', 'odprt', 'odprta', 'odprti', 
                  'okoli', 'on', 'onadva', 'one', 'oni', 'onidve', 'osem', 'osma', 'osmi', 'osmo', 'oz.', 
                  'pa', 'po', 'pod', 'pogosto', 'poleg', 'poln', 'polna', 'polni', 'polno', 'ponavadi',
                  'ponovno', 'potem', 'povsod', 'pozdravljen', 'pozdravljeni', 'prav', 'prava', 'prave', 'pravi', 'pravo',
                  'pribl.', 'precej', 'pred', 'preko', 'pri', 'približno', 'primer', 'proti', 'prva', 's',
                  'prvi', 'prvo','rajši','raje','rajš','rajše','rall' ,'ravno', 'redko', 'res', 'reč','riten.','riten', 'saj', 'sam', 'sama', 'same', 'sami', 
                  'samo', 'se', 'sebe', 'sebi', 'sedaj', 'sem', 'seveda', 'si', 'sicer', 'skoraj', 'skozi',
                  'smo', 'so', 'spet', 'sreda', 'srednja', 'srednji', 'srednje', 'sta', 'ste', 'stran', 'stvar', 'sva','svoja', 'svoj',
                  'svoje', 'svojim', 'svojih', 'sred', 'sredi','svojo','svoji','svojim','svojima', 'svojega',
                  'šel', 'šla', 'šli','šlo', 'še',
                  'ta', 'tak', 'taka', 'take', 'taki','takrat', 'tako', 'takoj', 'tam', 'te', 'tebe', 'tebi', 'treba',
                  'tega', 'težak', 'težka', 'težki', 'težko', 'ti', 'tista', 'tiste', 'tisti', 'tisto', 'tj.', 
                  'tja', 'to', 'toda', 'torej', 'tu', 'tudi', 'tukaj', 'tvoj', 'tvoja', 'tvoje', 'u', 'v', 
                  'vaju', 'vam', 'vas', 'vaš', 'vaša', 'vaše', 've', 'vedno', 'velik', 'velika', 'veliki', 
                  'veliko', 'vendar', 'ves', 'več', 'vi', 'vidva', 'vii', 'viii', 'vsa', 'vsaj', 'vsak',
                  'vsaka', 'vsakdo', 'vsake', 'vsaki', 'vsakomur', 'vse', 'vsega', 'vsi', 'vso', 'včasih', 
                  'včeraj', 'x', 'z', 'za','zame','zate' ,'zadaj', 'zadnji', 'zakaj', 'zaprta', 'zaprti', 'zaprto', 'zdaj', 'zelo',
                  'zunaj', 'če', 'često', 'čez', 'čigav', 'že', 'vsazga', 'tazga', 'jest', 'jast', 'svojga', 'mojga', 
                  'njenga','cresc','sost','sostenuto', 'riten', 'tempo', 'adagio', 'andante','allegro','piano', 'pianissimo','forte','fortissimo', 'maestoso']
    

try:
    lied_directory=os.scandir("/Users/vanessaninaborsan/Desktop/Študij/SMC/UPF_THESIS/_InterLied_first_draft/test_scores")

    for i in lied_directory:

        try:
            print("_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_")
            print("file:", i.name, "is looping.")
            print("_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_")

            #lyric_combined = string of lyric per interval, including stopwords.                 
            lyric_combined=[]
            #composer = composer of the score in which the interval is.
            composers=[]
            #title = title of the score in which the interval is.
            titles=[]
            #lyricist = lyricist of the score in which the interval is.
            lyricsts=[]
            #years = year of origin of the score in which the interval is. Has to be added manually afterwards! Only returns empty fields.
            years=[]
            #tonalities = tonalities of the score in which the interval is.
            tonalities=[]
            #alternative tonalities, which could substitute the one computed.
            alternatives=[]
            #voiceranges = voice range of the score in which the interval is.
            voiceranges=[]
            #meters = meters of the score in which the interval is.
            meters=[]
            #meter_changes = meter changes of the score in which the interval is.
            meter_changes=[]
            
            lyric_List=[]

            i_1=i.name
            i=i.path
            score=converter.parse(i)
            voice=score.parts[0]
            
            try:
                #COMPOSER
                comp=score.metadata.composer
                #TITLE
                title=score.metadata.title
                #YEAR
                year="None"
                #LYRICIST
                metadt=score.metadata.all()
                for element in metadt:
                    if(element[0]=='lyricist'):
                        lyricist=element[1]
                #KEY
                tonality=score.analyze('key')
                #ALTERNATIVE TOP 3 KEYS
                alt_t=[str(score) for score in tonality.alternateInterpretations[0:3]]
                #RANGE
                amb=analysis.discrete.analyzeStream(voice, 'ambitus')
                ambitus=amb.diatonic.name
                #METER CHANGE
                metChange = features.jSymbolic.ChangesOfMeterFeature(score)
                m_c = metChange.extract().vector
                
                #COMPOSER
                composers.append(str(comp))
                #TITLE
                titles.append(str(title))
                #YEAR
                years.append(str(year))
                #LYRICIST
                lyricsts.append(str(lyricist))
                #KEY
                tonalities.append(str(tonality))
                #ALTERNATIVE TOP 3 KEYS
                alternatives.append(str(alt_t))
                #RANGE
                voiceranges.append(str(ambitus))
                #METER CHANGE
                meter_changes.append(str(m_c))
                #METERS
                meters.append(str(meter))
                
                for y in m_c:
                    if y == 0:
                        m_c='NO'
                    elif y ==1:
                        m_c='YES'
                #METERS
                y= []
                lastTimeSignature=None
                
                for n in score.recurse().getElementsByClass('Note'):
                    thisTimeSignature = n.getContextByClass('TimeSignature')
                    if thisTimeSignature is not lastTimeSignature:
                        lastTimeSignature = thisTimeSignature
                        y.append(thisTimeSignature.ratioString)
                
                meter = set(y)


                bb=text.assembleLyrics(voice)
    #                     print([bb])

            except:
                bb="/"
                comp="not specified"
                composers.append(comp)
                title="not specified"
                titles.append(title)
                lyricst="not specified"
                lyricsts.append(lyricst)
                tonality="not specified"
                tonalities.append(tonality)
                alt_t="not specified"
                alternatives.append(alt_t)
                ambitus="not specified"
                voiceranges.append(ambitus)
                meter = "not specified"
                meters.append(meter)
                m_c="not specified"
                meter_changes.append(m_c)

                #Combined lyric for each interval.
            lyric=bb
            s_lyric_1ist=[]
            s_lyric_lower_list=[]
                
                #Delete unneccessary symbols and make all lower caps.
            s_lyric_1=re.sub("[,-?,.!@#$:;/*']", '', lyric)
            s_lyric_1=re.sub('[_]',' ', s_lyric_1)
            s_lyric_1ist+=s_lyric_1
            s_lyric_lower=str(s_lyric_1).lower()
            s_lyric_lower_list+=s_lyric_lower

            #Tokenize the lyric.
            tokens=nltk.word_tokenize(str(s_lyric_lower))
            lyric_combined+=[s_lyric_1]
            #Apply stopwords and filter them out.
            reduced_tokens = [w.lower() for w in tokens if w.lower() not in sw]

            #CREATE COMBINED LIST
            print("Creating a file...")   
#             for m in range (len(titles)):
#                 lyric_List+=[[titles[m], composers[m], years[m], lyricsts[m], tonalities[m], alternatives[m], 
#                            voiceranges[m], meter_changes[0], meters[m], lyric_combined[m], lyric_token_list[m]]]
            lyric_List+=[[titles, composers, years, lyricsts, tonalities, alternatives, 
                          voiceranges, meter_changes, meters, lyric_combined, reduced_tokens]]
                         
            #CREATE DATAFRAME
            df_lyric=pd.DataFrame(lyric_List)
            df_lyric = df_lyric.rename(columns=({0:"title" , 1:"composer", 2:"year", 3:"lyricist", 4:"key_s", 
                                                 5:"alternative_key_s", 6: "voice_range", 7:"meter_change", 8:"meter_s",
                                                 9:"lyric_string", 10:"lyric_tokens"}))
            
            df_lyric = df_lyric[["title", "composer", "year", "lyricist", "key_s", "alternative_key_s", 
                                 "voice_range", "meter_change", "meter_s", "lyric_string", "lyric_tokens"]]
    
            df_lyric.to_csv("lyric/_lyric_"+str(i_1) + "_combined"+".csv")

            print("---------------------------------------------------------------------------------------")
            print("***", i_1, "IS DONE***")
            print("---------------------------------------------------------------------------------------")
#             break

        except Exception as e: 
            print(e)
            print("-----------------------------------------!!!-------------------------------------------")
            print("ERROR occured.", i_1, "cannot be parsed.")
            print("-----------------------------------------!!!-------------------------------------------")
#             break

    print("*******************************************************************")
    print("******************A*L*L*********D*O*N*E****************************")
    print("*******************************************************************")

except:
    print("Directory cannot be found or is not compatible with the algorithms format. Check for possible errors in your database and try again.")
print(df_lyric)

_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_
file: .DS_Store is looping.
_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_
cannot find a format extensions for: /Users/vanessaninaborsan/Desktop/Študij/SMC/UPF_THESIS/_InterLied_first_draft/test_scores/.DS_Store
-----------------------------------------!!!-------------------------------------------
ERROR occured. .DS_Store cannot be parsed.
-----------------------------------------!!!-------------------------------------------
_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_
file: Kogoj_ce-soncna-se-raztezajo-vam-pota.mxl is looping.
_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_
Creating a file...
---------------------------------------------------------------------------------------
*** Kogoj_ce-soncna-se-raztezajo-vam-pota.mxl IS DONE***
---------------------------------------------------------------------------------------
_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_
file: Kogoj_ah-ne-verjemi.mxl is 

In [58]:
#Join desired csvs
path = r'/Users/vanessaninaborsan/Desktop/Študij/SMC/UPF_THESIS/lyric'                     # use your path
all_files = glob.glob(os.path.join(path, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent
df_from_each_file = (pd.read_csv(f) for f in all_files)
concatenated_df = pd.concat(df_from_each_file, ignore_index=True)
x=concatenated_df['lyric_tokens']
x=[d.split() for d in x]
# display(x)
dictionary = corpora.Dictionary(x)
print(dictionary)
#Total number of non-zeroes in the BOW matrix (sum of the number of unique words per document over the entire corpus).
print(dictionary.num_nnz)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in x ]
print(len(doc_term_matrix))
lda = gensim.models.ldamodel.LdaModel
num_topics=2
%time ldamodel = lda(doc_term_matrix,num_topics=num_topics,id2word=dictionary,passes=50,minimum_probability=0)
ldamodel.print_topics(num_topics=num_topics)

Dictionary(50 unique tokens: ["'anica']", "'anicadaj',", "'dedaj',", "'duši',", "'ie',"]...)
50
2
CPU times: user 71.5 ms, sys: 3.12 ms, total: 74.7 ms
Wall time: 79.6 ms


[(0,
  '0.073*"\'jezus\'," + 0.052*"\'bodi\'," + 0.031*"\'pota\'," + 0.031*"\'jezus\']" + 0.031*"\'moto\'," + 0.031*"\'najslajša\'," + 0.031*"\'našega\'," + 0.031*"\'noga\'," + 0.031*"\'ostro\'," + 0.031*"\'razpenjajo\',"'),
 (1,
  '0.040*"\'tolažbe\'," + 0.040*"\'vrata\'," + 0.040*"\'mo\'," + 0.040*"\'tej\'," + 0.040*"\'kličem\'," + 0.040*"\'ječe\'," + 0.040*"\'odkleni\'," + 0.024*"\'ie\'," + 0.024*"\'pridi\'," + 0.024*"[\'verjemi\',"')]

In [59]:
doc_term_matrix = [dictionary.doc2bow(doc) for doc in df_lyric['lyric_tokens'] ]
print(len(doc_term_matrix))

1


In [60]:
lda = gensim.models.ldamodel.LdaModel

In [61]:
num_topics=2
%time ldamodel = lda(doc_term_matrix,num_topics=num_topics,id2word=dictionary,passes=50,minimum_probability=0)

CPU times: user 42.6 ms, sys: 4.5 ms, total: 47.1 ms
Wall time: 53 ms


In [62]:
ldamodel.print_topics(num_topics=num_topics)

[(0,
  '0.020*"\'jezus\'," + 0.020*"\'edini\'," + 0.020*"\'najslajša\'," + 0.020*"\'moto\'," + 0.020*"\'jezus\']" + 0.020*"\'noga\'," + 0.020*"\'žaiosti\'," + 0.020*"[\'verjemi\'," + 0.020*"\'pota\'," + 0.020*"\'bodi\',"'),
 (1,
  '0.020*"\'jezus\'," + 0.020*"\'edini\'," + 0.020*"\'najslajša\'," + 0.020*"\'moto\'," + 0.020*"\'jezus\']" + 0.020*"\'noga\'," + 0.020*"\'žaiosti\'," + 0.020*"[\'verjemi\'," + 0.020*"\'pota\'," + 0.020*"\'bodi\',"')]

In [63]:
lda_display = pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary, sort_topics=False, mds='mmds')
pyLDAvis.display(lda_display)

In [64]:
lda_corpus = ldamodel[doc_term_matrix]

In [65]:
[doc for doc in lda_corpus]

[[(0, 0.5), (1, 0.5)]]

In [66]:
scores = list(chain(*[[score for topic_id,score in topic] \
                      for topic in [doc for doc in lda_corpus]]))

threshold = sum(scores)/len(scores)
print(threshold)

0.5


In [67]:
cluster1 = [j for i,j in zip(lda_corpus,df.index) if i[0][1] > threshold]
cluster2 = [j for i,j in zip(lda_corpus,df.index) if i[1][1] > threshold]
# cluster3 = [j for i,j in zip(lda_corpus,df.index) if i[2][1] > threshold]
# cluster4 = [j for i,j in zip(lda_corpus,df.index) if i[3][1] > threshold]
# cluster5 = [j for i,j in zip(lda_corpus,df.index) if i[4][1] > threshold]

print(len(cluster1))
print(len(cluster2))
# print(len(cluster3))
# print(len(cluster4))
# print(len(cluster5))

NameError: name 'df' is not defined

In [68]:
df.iloc[cluster1]

NameError: name 'df' is not defined

In [ ]:
df.iloc[cluster2]


In [ ]:
num=0
df1=df.iloc[cluster1]
df2=df.iloc[cluster2]
lists=[]
topics=[]

for index_num in df1.iterrows():
    index_num=index_num[0]
    lists.append(index_num)

for i in df.iterrows():
    index_n=i[0]
    if index_n in lists:
        t1="T1"
        topics.append("T1")
#         print("T1")
    else:
        t2="T2"
        topics.append("T2")
#         print("T2")

df_w_topics=df
df_w_topics["predicted_topic"]=topics
df_w_topics